In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import rmsprop
from keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint
        

Using TensorFlow backend.


# Loading Tranning Data 

In [2]:
data_data = pd.read_csv('/home/nitin/Desktop/keras-regression-master/database/train/train_100k.csv') #'../database/train/train_100k.csv'


In [3]:
data_data.head(2)

,id,x0,y0,x1,y1,x2,y2,x3,y3,x4,...,x5,y5,x6,y6,x7,y7,x8,y8,x9,y9
0,0,29.379006,-177.906597,66.239362,-143.49507,-107.178287,-338.177166,22.166375,-185.843612,84.694131,...,136.901770,-81.681363,-84.118333,-334.626033,-100.681489,-336.766765,-37.043301,-253.037134,0.695548,-211.044600
1,1,-42.192600,116.207765,81.428046,-94.91560,-68.070200,154.431135,87.454642,-119.361432,-86.614036,...,-40.894683,123.924804,111.751223,-139.521058,15.587590,-0.594559,-19.313860,70.677833,-53.806709,133.994773


# Loading Ground Truth for Tranning Data 


In [4]:
data_labels = pd.read_csv("/home/nitin/Desktop/keras-regression-master//database/train/train_100k.truth.csv") # '../database/train/train_100k.truth.csv'


In [5]:
data_labels.head(2)

,id,slope,intercept
0,0,1.132627,-219.723905
1,1,-1.847354,40.482329


# Loading Test data set 

In [6]:
data_test = pd.read_csv("/home/nitin/Desktop/keras-regression-master/database/test/test_100k.csv") # '../database/test/test_100k.csv'
 

In [7]:
data_test.head(2)

,id,x0,y0,x1,y1,x2,y2,x3,y3,x4,...,x5,y5,x6,y6,x7,y7,x8,y8,x9,y9
0,0,71.222322,108.068811,-18.007281,237.215087,-3.841226,204.741403,-58.849720,260.483835,-52.056002,...,11.751514,200.906386,-72.994373,288.208877,93.297193,69.952999,-47.006169,262.735790,40.499569,159.883677
1,1,-3.214690,12.107857,-25.870974,48.638382,58.614339,-52.683957,-7.681873,10.727864,59.550048,...,-22.436669,26.141211,-41.874875,61.856665,5.690588,2.823426,13.122010,-11.562005,-63.664182,76.761295


In [8]:
train_data = data_data.values
train_data[0]

array([   0.        ,   29.37900571, -177.90659732,   66.23936185,
       -143.49507023, -107.17828725, -338.17716559,   22.16637465,
       -185.84361204,   84.6941314 , -130.54795596,  136.90176958,
        -81.68136257,  -84.11833255, -334.62603307, -100.68148866,
       -336.76676483,  -37.04330138, -253.03713404,    0.69554829,
       -211.04459974])

In [9]:
train_labels = data_labels.values
train_labels[0]

array([   0.        ,    1.13262672, -219.72390509])

In [10]:
td = train_data[:, 1:]
td[0]


array([  29.37900571, -177.90659732,   66.23936185, -143.49507023,
       -107.17828725, -338.17716559,   22.16637465, -185.84361204,
         84.6941314 , -130.54795596,  136.90176958,  -81.68136257,
        -84.11833255, -334.62603307, -100.68148866, -336.76676483,
        -37.04330138, -253.03713404,    0.69554829, -211.04459974])

In [11]:
tl = train_labels[:, 1:]
tl[0]

array([   1.13262672, -219.72390509])

In [12]:
test_data = data_test.values[:, 1:]
test_data[0]

array([ 71.2223217 , 108.06881074, -18.00728057, 237.21508658,
        -3.84122624, 204.74140346, -58.84972014, 260.48383521,
       -52.056002  , 271.4469372 ,  11.75151377, 200.90638634,
       -72.99437307, 288.20887701,  93.29719257,  69.95299883,
       -47.00616922, 262.7357902 ,  40.4995692 , 159.8836771 ])

In [13]:
model = Sequential()
model.add(Dense(50, input_dim=td.shape[1], activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(2, activation='linear'))

opt = keras.optimizers.rmsprop(learning_rate=0.01)




model.compile(optimizer=opt, loss='mse', metrics=['acc', 'mae'])




    

<IPython.core.display.Javascript object>

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                1050      
_________________________________________________________________
dense_2 (Dense)              (None, 40)                2040      
_________________________________________________________________
dense_3 (Dense)              (None, 30)                1230      
_________________________________________________________________
dense_4 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_5 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 22        
Total params: 5,172
Trainable params: 5,172
Non-trainable params: 0
____________________________________________________

# Setting Callback Functions

In [15]:
earlystop = EarlyStopping(patience=10)

In [16]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [17]:
filepath = "saved-model-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max')

In [18]:
callbacks = [earlystop,learning_rate_reduction,checkpoint]

In [19]:
history = model.fit(td,tl, epochs=300, verbose=2,callbacks=callbacks,validation_split=0.3)

Train on 70000 samples, validate on 30000 samples
Epoch 1/300
 - 3s - loss: 1390.1402 - acc: 0.9593 - mae: 16.7814 - val_loss: 1168.2797 - val_acc: 0.9633 - val_mae: 16.6197

Epoch 00001: saving model to saved-model-0.96.hdf5
Epoch 2/300
 - 3s - loss: 474.0338 - acc: 0.9767 - mae: 10.2971 - val_loss: 336.4472 - val_acc: 0.9804 - val_mae: 9.9249

Epoch 00002: saving model to saved-model-0.98.hdf5
Epoch 3/300
 - 3s - loss: 354.9728 - acc: 0.9803 - mae: 9.0205 - val_loss: 257.2424 - val_acc: 0.9817 - val_mae: 7.5761

Epoch 00003: saving model to saved-model-0.98.hdf5
Epoch 4/300
 - 3s - loss: 296.7601 - acc: 0.9810 - mae: 8.4017 - val_loss: 314.8060 - val_acc: 0.9836 - val_mae: 8.6061

Epoch 00004: saving model to saved-model-0.98.hdf5
Epoch 5/300
 - 3s - loss: 272.8698 - acc: 0.9812 - mae: 8.0752 - val_loss: 258.0281 - val_acc: 0.9780 - val_mae: 8.0310

Epoch 00005: saving model to saved-model-0.98.hdf5
Epoch 6/300
 - 3s - loss: 252.3715 - acc: 0.9821 - mae: 7.7645 - val_loss: 158.7654 -

 - 3s - loss: 32.1499 - acc: 0.9903 - mae: 2.9921 - val_loss: 45.4607 - val_acc: 0.9907 - val_mae: 3.1719

Epoch 00044: saving model to saved-model-0.99.hdf5
Epoch 45/300
 - 3s - loss: 32.1358 - acc: 0.9904 - mae: 2.9917 - val_loss: 45.5416 - val_acc: 0.9907 - val_mae: 3.1721

Epoch 00045: saving model to saved-model-0.99.hdf5
Epoch 46/300
 - 3s - loss: 32.1402 - acc: 0.9904 - mae: 2.9918 - val_loss: 45.4629 - val_acc: 0.9908 - val_mae: 3.1718

Epoch 00046: saving model to saved-model-0.99.hdf5
Epoch 47/300
 - 3s - loss: 32.1548 - acc: 0.9903 - mae: 2.9916 - val_loss: 45.5007 - val_acc: 0.9907 - val_mae: 3.1720

Epoch 00047: saving model to saved-model-0.99.hdf5
Epoch 48/300
 - 3s - loss: 32.1573 - acc: 0.9903 - mae: 2.9918 - val_loss: 45.5222 - val_acc: 0.9907 - val_mae: 3.1735

Epoch 00048: saving model to saved-model-0.99.hdf5
Epoch 49/300
 - 3s - loss: 32.1565 - acc: 0.9903 - mae: 2.9909 - val_loss: 45.4453 - val_acc: 0.9908 - val_mae: 3.1731

Epoch 00049: saving model to saved-mod

In [20]:
Accuracy = history.history['acc']
Val_Accuracy = history.history['val_acc']



In [21]:
Accuracy =  Accuracy.pop()
Val_Accuracy = Val_Accuracy.pop()
print("Accuracy",Accuracy)
print("Val_Accuracy ",Val_Accuracy) 

Accuracy 0.99032855
Val_Accuracy  0.9907666444778442


In [22]:
result_test = model.predict(test_data)

In [23]:
result_test_dataframe = pd.DataFrame(result_test, columns=['slope', 'intercept'])

In [24]:
result_test_dataframe.index.name = 'id'

In [25]:
result_test_dataframe.to_csv("result.csv")